In [1]:
pip install csv

ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


In [2]:
import csv

Открываю вручную размеченный корпус. Так называемы Золотой Стандарт. Была выбрана экономическая статья, так как в ней есть несколько аббревиатур, например ООН, также были слова с дефисом. Разметка - Universal Dependencies

In [3]:
with open ('hand_rasmetka.csv', 'r', encoding = 'utf-8') as tt:
  reader = csv.reader(tt)
  ideal = []
  for row in reader:
    ideal.append(row)

In [4]:
with open ('text_for_dz.txt', 'r', encoding = 'utf-8') as pos:
    text = pos.read()

Я выбрала такие POS теггеры: natasha, spacy, mystem

In [6]:
!pip install -U spacy>=3.0

In [7]:
!python -m spacy download ru_core_news_md

     |████████████████████████████████| 42.7 MB 2.5 kB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 8.2 MB 48.9 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [8]:
import spacy

In [9]:
nlp = spacy.load('ru_core_news_md')

In [10]:
document = nlp(text)

In [11]:
result_spacy = []
for token in document:
  result_spacy.append([token.text, token.pos_])  

Далее идет анализ mystem. У меня получилось так, что mystem не работает в Google Colab, поэтому я прогнала текст в Jupyter и сделала выгрузку в test.csv. Я не сделала все дз в Jupyter, потому что в нем у меня работает spacy. 

In [ ]:
pip install pymystem3

In [ ]:
from pymystem3 import Mystem
m = Mystem()

In [ ]:
ana = m.analyze(text)

In [ ]:
result_mystem = []
for word in ana:
    if 'analysis' in word:
        gr = word['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        result_mystem.append([word['text'],pos])


In [ ]:
import pandas as pd
df = pd.DataFrame(result_mystem)
df.to_csv('test.csv', index=False, header=False)

NameError: ignored

In [12]:
with open ('test.csv', 'r', encoding = 'utf-8') as tt2:
  reader = csv.reader(tt2)
  result_of_mystem = []
  for row in reader:
    result_of_mystem.append(row)

In [13]:
pip install natasha

     |████████████████████████████████| 34.4 MB 29 kB/s 
     |████████████████████████████████| 41 kB 139 kB/s 
     |████████████████████████████████| 49 kB 7.4 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=0e6089ea6499e5604b87c0260ced3e72f96aeeb74955346c4e46f25d9de9e46f
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
Successfully built intervaltree
  Attempting uninstall: intervaltree
    Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [14]:
from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, Doc
segmenter = Segmenter()


emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [15]:
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsMorphTagger,
    Doc)

In [16]:
segmenter = Segmenter()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [17]:
doc = Doc(text)
doc.segment(segmenter) 
doc.tag_morph(morph_tagger)

In [18]:
result_natasha = []
for token in doc.tokens:
    result_natasha.append([token.text, token.pos])

Далее я привожу все к единому стандарту, а именно Universal Dependencies

In [19]:
for e in range(len(result_of_mystem)):
  if result_of_mystem[e][1] == 'A':
    result_of_mystem[e][1] = 'ADJ'
  if result_of_mystem[e][1] == 'CONJ':
    result_of_mystem[e][1] = 'CCONJ'
  if result_of_mystem[e][1] == 'S':
    result_of_mystem[e][1] = 'NOUN'
  if result_of_mystem[e][1] == 'V':
    result_of_mystem[e][1] = 'VERB'
  if result_of_mystem[e][1] == 'ANUM':
    result_of_mystem[e][1] = 'NUM'
  if result_of_mystem[e][1] == 'PR':
    result_of_mystem[e][1] = 'ADP'
  if result_of_mystem[e][1] == 'APRO':
    result_of_mystem[e][1] = 'PRON'
  if result_of_mystem[e][1] == 'PART':
    result_of_mystem[e][1] = 'x'

Accuracy Mystem

In [20]:
result_mystem_hand = []
for c1 in ideal:
  for x1 in result_of_mystem:
    if c1 == x1:
      result_mystem_hand.append(c1)
      break
correct_mystem = (len(result_mystem_hand))

In [21]:
accuracy_mystem = correct_mystem/236*100

Accuracy spacy

In [22]:
result_spacy_hand = []
for c2 in ideal:
  for x2 in result_spacy:
    if c2 == x2:
      result_spacy_hand.append(c2)
      break
correct_spacy = (len(result_spacy_hand))

In [27]:
accuracy_spacy = correct_spacy/236*100

Accuracy Natasha

In [24]:
result_natasha_hand = []
for c4 in ideal:
  for x4 in result_natasha:
    if c4 == x4:
      result_natasha_hand.append(c4)
      break
correct_natasha = (len(result_natasha_hand))

In [29]:
accuracy_natasha = correct_natasha/236*100

Лучшим теггером получился spacy

In [30]:
print(accuracy_mystem)
print(accuracy_natasha)
print(accuracy_spacy)

75.42372881355932
82.20338983050848
83.47457627118644


In [ ]:
from spacy.matcher import Matcher

Я выбрала такой шаблон: прилагательное + существительное, предлог + существительное, наречие + прилагательное

In [ ]:
list_of_rules = [
    ['ADJ','NOUN'],
    ['ADP','NOUN'],
    ['ADV','ADJ']]

In [ ]:
rules = [[{"POS": i} for i in j] for j in list_of_rules]

In [ ]:
list_of_rules = [
    ['ADJ','NOUN'],
    ['ADP','NOUN'],
    ['ADV','ADJ']
]

rules = [[{"POS": i} for i in j] for j in list_of_rules]

matcher = Matcher(nlp.vocab)
matcher.add("rules", [*rules], on_match=None)

In [ ]:
matches = matcher(document)

In [ ]:
print([document[start:end].text for _, start, end in matches])

['целым рядом', 'объективных факторов', 'экономического развития', 'человеческого общества', 'второго тысячелетия', 'извечной мечты', 'международного общежития', 'от бесконечности', 'мирные условия', 'поступательное продвижение', 'по пути', 'экономического прогресса', 'от страха', 'за будущее', 'всеобщей организации', 'Атлантическая партия', '14 августа', '1941 г', 'межсоюзной конференции', '24 сентября', '1941 г', 'чрезвычайно важная', 'важная задача', 'миролюбивыми государствами', 'для организации', 'международных отношений', 'послевоенного устройства', 'межправительственным документом', 'в годы', 'мировой войны', 'международной организации', 'о дружбе', '4 декабря', '1941 г', 'справедливого мире', 'новой организацией', 'международных отношений', 'демократических стран', 'прочный союз', 'При создании', 'решающим моментом', 'международному праву', 'Союзных государств', 'В ходе', '25 апреля', '1945 г', '26 июня', '1945 г']


К прошлому домашнему заданию я предложила бы такой шаблон: не + VERB, так как часто словосочетание "не нравится" не учитывается, а если еще убрать все стоп-слова, то частица "не" совсем пропадает. 
Также предложила бы шаблон : "очень + VERB" или "очень + ADJ", чтобы было различие между позитивным отзывом и нейтральным. Так как "очень" придает предложению эмоциональной окраски. 